In [1]:
!pip install ../

Processing c:\cygwin64\home\evger\thesis
  Created wheel for backtesting-hft: filename=backtesting_hft-1.0.0-py3-none-any.whl size=36508 sha256=a6d15329d0d439bd40a5060cf559408ab1f71c91c1e3c167a5d1545cea183cef
  Stored in directory: C:\cygwin64\tmp\pip-ephem-wheel-cache-t0m6y347\wheels\c4\e2\a2\12f7ab9f08c449a62bb02647df45a7056114c556f36fa34ffb
Successfully built backtesting-hft
  Attempting uninstall: backtesting-hft
    Found existing installation: backtesting-hft 1.0.0
    Uninstalling backtesting-hft-1.0.0:
      Successfully uninstalled backtesting-hft-1.0.0


In [2]:
import pandas as pd
import glob
import os

In [3]:
dst_dir = 'time-sampled'
pairs = zip(glob.glob(f'{dst_dir}/orderbook_*'), glob.glob(f'{dst_dir}/trade_*'))

In [ ]:
from hft.backtesting.strategy import Strategy
from hft.backtesting.backtest import BacktestOnSample
from hft.backtesting.readers import OrderbookReader

from typing import List, Dict, Tuple, Union, Callable
from hft.utils.data import OrderBook, Trade
from hft.backtesting.data import OrderStatus, OrderRequest

from hft.units.metrics.instant import *
from hft.units.metrics.time import TradeMetric



tick_counter = 0
names = ['vwap', 'liquidity-spectrum', 'trade-metric-60', 'hayashi-yoshido', 'lipton']

class RLStrategy(Strategy):
    
    def get_state(self):
#         vwap1 = np.fromiter(self.metrics_map['vwap1'].latest.values(), dtype=np.float) # np array [2, 3, 2]
#         liq   = np.fromiter(self.metrics_map['liquidity-spectrum'].latest.values(), dtype=np.float) # np array [2, 3]
#         trades = np.fromiter(self.metrics_map['trade-metric-60'].latest.values(), dtype=np.float) # np array [2, 2] ???
#         hy = np.fromiter(self.metrics_map['hayashi-yoshido'].latest.values(), dtype=np.float) # np array [2, 2]
#         lipton = np.fromiter(self.metrics_map['lipton'].latest.values(), dtype=np.float) # np array 
        # todo: may be without list() for dict_values collection?
        items = list(map(lambda name: np.fromiter(self.metrics_map[name].latest.values(), dtype=np.float), names))
        return items
        
    
    def define_orders(self, row: Union[Trade, OrderBook],
                    statuses: List[OrderStatus],
                    memory: Dict[str, Union[Trade, OrderBook]]) -> List[OrderRequest]:
        if tick_counter + 1 % 3000 == 0:
            obs = self.get_state()
            obs = np.stack(obs)
            
            orders = []
        else:
            orders = []
            
        tick_counter += 1
        return orders
        


def init_simulation(orderbook_file, trade_file):
    
    vwap1 = VWAP_volume(int(2.5e5), name='vwap1')
    vwap2 = VWAP_volume(int(1e6), name='vwap2')
    liq = LiquiditySpectrum()
    
    
    trade_metric = TradeMetric([
        ('quantity', lambda x: len(x)),
        ('total', lambda x: sum(x))
    ], seconds=60)

    hy = HayashiYoshido()
    lipton = Lipton(hy.name)
    
    reader = OrderbookReader(orderbook_file, trade_file, nrows=None, is_precomputed=True)
    strategy = RLStrategy([vwap1, vwap2, liq], [hy], [trade_metric], composite_metrics=[lipton], initial_balance=0.0)
    backtest = BacktestOnSample(reader, strategy, delay=300)
    backtest.run()